<a href="https://colab.research.google.com/github/SadehYuval/MiniProjectDataManagment/blob/main/MiniProjectDataManagment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def algorithm_1(items, K, num_categories, lower_constraints, upper_constraints):
    counter = 0
    # Initialize the output list L
    L = []

    # Initialize the counts of per-category selected items C
    C = [0] * num_categories

    slack = K - sum(lower_constraints)

    while len(L) < K:
        x = items[counter]
        counter += 1
        i = x.category

        # Check if the lower constraint is satisfied for category i
        if C[i] < lower_constraints[i]:
            L.append(x)
            C[i] += 1
        # Check if upper constraint can be relaxed
        elif C[i] < upper_constraints[i] and slack > 0:
            L.append(x)
            C[i] += 1
            slack -= 1

    return L



In [2]:
import heapq
import math

def algorithm_2(items, K, N, num_categories, lower_constraints, upper_constraints, items_per_category):

    L = []

    # Array holding how many items we accepted from each category
    C = [0] * num_categories

    # Array holding how many items we've seen from each category
    M = [0] * num_categories

    # Per category warmup
    R = [math.floor(n / math.e) for n in items_per_category]

    T = [[] for _ in range(num_categories)]

    slack = K - sum(lower_constraints)

    # Independent warmup
    r = math.floor(N / math.e)

    T_independent = []

    def num_feasible_items():
      return sum(items_per_category[i] - M[i] for i in range(len(items_per_category)))

    counter = 0

    while len(L) < K:
        x = items[counter]
        counter += 1
        i = x.category

        # Check if category-independent warm-up is needed
        if sum(M) < r:
            if len(T_independent) < slack:
              heapq.heappush(T_independent, x)
            elif T_independent[0].score < x.score:
              heapq.heappop(T_independent)
              heapq.heappush(T_independent, x)

        # Check if category-specific warm-up is needed
        if M[i] < R[i]:
            if len(T[i]) < lower_constraints[i]:
              heapq.heappush(T[i], x)
            elif T[i][0].score < x.score:
              heapq.heappop(T[i])
              heapq.heappush(T[i], x)

        elif (C[i] < lower_constraints[i] and x.score > T[i][0].score) or (items_per_category[i] - M[i] == lower_constraints[i] - C[i]):
            heapq.heappop(T[i])
            L.append(x)
            C[i] += 1
        elif (sum(M) >= r and x.score > T_independent[0].score and C[i] < upper_constraints[i] and slack > 0):
            heapq.heappop(T_independent)
            L.append(x)
            C[i] += 1
            slack -= 1
        elif C[i] < upper_constraints[i] and num_feasible_items() == K - len(L):
            L.append(x)
            C[i] += 1
            slack -= 1

        # Increment the count of seen items for the category
        M[i] += 1

    return L


In [5]:
import heapq
import math

def algorithm_3(items, K, N, num_categories, lower_constraints, upper_constraints, items_per_category):
    L = []

    D = [[] for _ in range(num_categories)]

    C = [0] * num_categories

    M = [0] * num_categories

    R = [math.floor(n / math.e) for n in items_per_category]

    T = [[] for _ in range(num_categories)]

    # Initialize the number of unsatisfied categories
    u = num_categories - sum(1 for i in range(num_categories) if lower_constraints[i] == 0)

    # w - Number of deferred items overall
    w = 0
    counter = 0

    while (u > 0) or (w < K):
        x = items[counter]
        counter += 1
        i = x.category

        if M[i] < R[i]:
            if len(T[i]) < lower_constraints[i]:
              heapq.heappush(T[i], x)
            elif T[i][0].score < x.score:
              heapq.heappop(T[i])
              heapq.heappush(T[i], x)
        elif (C[i] < lower_constraints[i]) and T[i] and x.score > T[i][0].score:
            C[i] += 1
            if T[i]:
              heapq.heappop(T[i])
            if (lower_constraints[i] > 0) and (C[i] == lower_constraints[i]):
                u -= 1

        # Add the item to its category's deferred list if its score is higher than the minimum element
        if len(D[i]) < upper_constraints[i]:
            heapq.heappush(D[i], x)
        # In the case the list has reached it's maximum capacity, replace the minimum if the current score is higher
        elif x.score > D[i][0].score:
            heapq.heappop(D[i])
            heapq.heappush(D[i], x)

        M[i] += 1

        # Update the total number of deferred items
        w = sum(len(D[i]) for i in range(num_categories))

    # Create a sorted list from the total deferred items
    W = []
    for i in range(num_categories):
        W.extend(D[i])

    W.sort(key=lambda x: x.score, reverse=True)


    # Compute L with Algorithm 1 on W (sorted by score)
    L = algorithm_1(W, K, num_categories, lower_constraints, upper_constraints)

    return L



In [6]:
import heapq
class Item:
    def __init__(self, category, score):
        self.category = category
        self.score = score

    def __lt__(self, other):
        return self.score < other.score

    def __le__(self, other):
        return self.score <= other.score

    def __eq__(self, other):
        return self.score == other.score

    def __ne__(self, other):
        return self.score != other.score

    def __gt__(self, other):
        return self.score > other.score

    def __ge__(self, other):
        return self.score >= other.score

items_list = [
    Item(category=0, score=6),
    Item(category=1, score=4),
    Item(category=0, score=1),
    Item(category=0, score=8),
    Item(category=1, score=2),
    Item(category=0, score=3),
    Item(category=1, score=1),
    Item(category=0, score=2),
    Item(category=1, score=9),
    Item(category=1, score=5),
    Item(category=0, score=7),
    Item(category=1, score=5)
]

# algorithm_2(items, K, N, num_categories, lower_constraints, upper_constraints, items_per_category)
output = algorithm_3(items_list, 3, 12, 2, [1, 1], [2, 2], [6, 6])
for item in output:
  print(item.score)


9
8
6
